In [8]:
import numpy as np
import time

from collections import namedtuple

In [2]:
'''
struct to storage order price and volume to trade

Usage:
a = Order(price=1, volume=1)
'''

Order = namedtuple('order', 'id price volume')

In [3]:
class dynamic_order:
    '''
    extension of Order, used in making deals
    '''
    def __init__(self, order):
        assert isinstance(order, Order)
        self.id = order.id
        self.price = order.price
        self.volume = order.volume
    @property
    def show(self):
        print('id : %d' % self.id)
        print('price : %d' % self.price)
        print('volume : %d' % self.volume)

In [4]:
class Trader:
    '''
    A primary class to define an investor
    '''
    def __init__(self, id):
        self.id = id
        self.cash = 0 # 现金
        self.cash_history = []
        self.asset = 0 # 总资产
        self.asset_history = []
        self.stock = 0 # 持股
        self.stock_history = []
        self.trade_history = [] # 交易记录
        self.price = 0
        self.order = Order(id=self.id,
                           price=0,
                           volume=0)
        
    def gen_order(self):
        self.order = Order(id=self.id,
                           price=np.random.randint(-10, 10), 
                           volume=np.random.randint(1, 5))
        
    def update(self, price, vol):
        vol *= np.sign(self.order.price)
        price = abs(price)
        self.cash -= price * vol
        self.cash_history.append(self.cash)
        self.stock += vol
        self.stock_history.append(self.stock)
        self.trade_history.append((price, vol))
        self.price = price
        self.asset = self.cash + self.stock * self.price
        self.asset_history.append(self.asset)
        
    @property
    def show(self):
        print('id : %d, cash : %d, stock : %d, stock_price : %d, asset : %d' % 
             (self.id, self.cash, self.stock, self.price, self.asset))
        
#         print('asset : %d' % self.asset)
#         print('stock : %d' % self.stock)
        

In [5]:
def make_deals(orders):
    buy_list = [dynamic_order(order) for order in orders if order.price>0]
    sell_list = [dynamic_order(order) for order in orders if order.price<0]

    buy_list = sorted(buy_list, key=lambda x:(-x.price, x.id))
    sell_list = sorted(sell_list, key=lambda x:(abs(x.price), x.id))
    deal_result = np.zeros(len(orders), int)
    price = 0
    if len(buy_list)==0 or len(sell_list)==0:
        return price, deal_result
    while sell_list[0].price+buy_list[0].price>=0:
        if sell_list[0].volume>=buy_list[0].volume:
            sell_list[0].volume -= buy_list[0].volume
            deal_result[buy_list[0].id] += buy_list[0].volume
            deal_result[sell_list[0].id] += buy_list[0].volume
            if sell_list[0].volume==buy_list[0].volume:
                price = (buy_list[0].price + abs(sell_list[0].price)) / 2
            else:
                price = abs(sell_list[0].price)
            _ = buy_list.pop(0)
        else:
            buy_list[0].volume -= sell_list[0].volume
            deal_result[buy_list[0].id] += sell_list[0].volume
            deal_result[sell_list[0].id] += sell_list[0].volume
            price = buy_list[0].price
            _ = sell_list.pop(0)           
        price = abs(price)
        if len(buy_list)==0 or len(sell_list)==0:
            break
    return price, deal_result

In [6]:
investors = [Trader(i) for i in range(200)]

In [13]:
orders = []
for investor in investors:
    investor.gen_order()
    orders.append(investor.order)
    
orders

[order(id=0, price=-4, volume=4),
 order(id=1, price=-1, volume=3),
 order(id=2, price=7, volume=3),
 order(id=3, price=-6, volume=4),
 order(id=4, price=3, volume=1),
 order(id=5, price=-2, volume=3),
 order(id=6, price=5, volume=2),
 order(id=7, price=-4, volume=2),
 order(id=8, price=5, volume=2),
 order(id=9, price=-10, volume=1),
 order(id=10, price=3, volume=3),
 order(id=11, price=5, volume=4),
 order(id=12, price=-10, volume=1),
 order(id=13, price=-6, volume=3),
 order(id=14, price=-1, volume=1),
 order(id=15, price=-6, volume=3),
 order(id=16, price=5, volume=3),
 order(id=17, price=-3, volume=1),
 order(id=18, price=-2, volume=3),
 order(id=19, price=-10, volume=1),
 order(id=20, price=0, volume=2),
 order(id=21, price=-2, volume=2),
 order(id=22, price=0, volume=2),
 order(id=23, price=-8, volume=2),
 order(id=24, price=-2, volume=3),
 order(id=25, price=4, volume=3),
 order(id=26, price=-10, volume=1),
 order(id=27, price=-4, volume=4),
 order(id=28, price=-2, volume=3),
 

In [9]:
start = time.time()
for step in range(3000):
    orders = []
    for investor in investors:
        investor.gen_order()
        orders.append(investor.order)

    price, deal_result = make_deals(orders)
    for i,investor in enumerate(investors):
        investor.update(price=price, vol=deal_result[i])
        
print('用时:', time.time()-start)

用时: 11.623684644699097


In [10]:
len(orders)

200

In [11]:
for investor in investors:
    investor.show

id : 0, cash : -268, stock : 63, stock_price : 5, asset : 47
id : 1, cash : 16, stock : 6, stock_price : 5, asset : 46
id : 2, cash : 950, stock : -193, stock_price : 5, asset : -15
id : 3, cash : -455, stock : 100, stock_price : 5, asset : 45
id : 4, cash : -259, stock : 94, stock_price : 5, asset : 211
id : 5, cash : 273, stock : -27, stock_price : 5, asset : 138
id : 6, cash : -271, stock : 80, stock_price : 5, asset : 129
id : 7, cash : 814, stock : -155, stock_price : 5, asset : 39
id : 8, cash : -49, stock : 19, stock_price : 5, asset : 46
id : 9, cash : 131, stock : -23, stock_price : 5, asset : 16
id : 10, cash : 4, stock : 1, stock_price : 5, asset : 9
id : 11, cash : 395, stock : -73, stock_price : 5, asset : 30
id : 12, cash : -576, stock : 103, stock_price : 5, asset : -61
id : 13, cash : 106, stock : -36, stock_price : 5, asset : -74
id : 14, cash : -158, stock : 64, stock_price : 5, asset : 162
id : 15, cash : -419, stock : 92, stock_price : 5, asset : 41
id : 16, cash : 

In [80]:
import time
a = time.time()

In [82]:
b = time.time()-a
print(b)

14.392970323562622
